# **Computational Creativity Project**

---

### **Recipe generation and image generation based on the generated recipe using LSTM and Big Sleep**

Samil Enes Basbug, Student ID: 210617146


## Important information about this notebook:

In this project, I built a text generation to generate chicken, meat, and fish recipes. Then, I utilized the Big Sleep model to generate images related to the recipes my model created. Please do not skip any line of code(run each piece of code respectively), I have already commented the training part out. The notebook is ready to be tested.

1.   I built and deployed an web app, using Streamlit (is an open-source app framework), generating food recipes. I recommend you to visit this web app as well :)  https://share.streamlit.io/enesbasbug/food_recipe_generator/main/app.py

 (The only model embedded in the web app was Recipe(text) generation model. I could not work with Big Sleep locally since CUDA only works with Nvidia graphic card I did not have. Therefore, even though I integrated Big Sleep to my Text Generation model on my computer, Big Sleep did not work.) 
2.   You may also visit my GitHub https://github.com/enesbasbug/Food_Recipe_Generator repository to see many texts/images/gifs examples of my project.



What you will see in this notebook respectively:
*   Installing the OpenAI's CLIP and a BigGAN model, also known as Big Sleep.
*   Importing the libraries necessary.
*   Loading the pre-trained model for text/recipe generator.
*   Loading the data, data cleaning and creating the files contain new data, which will be used by the text generator model
*   Reading the data
*   The common functions and variables that we will use
*   Generating the Text/Recipe (You will see there are 3 similar, in order, blocks of codes to generate the recipes since there are 3 specific pre-trained models for fish, meat and chicken dishes.) You may execute and try them by yourself
*   Determining the ingredients in the recipes
*   Image generation from the recipe/text using Big Sleep. You may execute and try it by yourself.





# Installing the Big Sleep Model for image generation and importing the libraries necessary
We will generate images using the generated text/recipes as inputs in last step

In [ ]:
!pip install big-sleep

     |████████████████████████████████| 1.4 MB 8.3 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 87 kB 8.0 MB/s 
     |████████████████████████████████| 132 kB 83.7 MB/s 
     |████████████████████████████████| 8.7 MB 55.7 MB/s 
     |████████████████████████████████| 79 kB 8.7 MB/s 
     |████████████████████████████████| 138 kB 56.9 MB/s 
     |████████████████████████████████| 127 kB 90.0 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=147effed85b35f0b6021eaffd528269fc87d6cfba64223299a1b398ba8d5d287
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are instal

In [ ]:
import pandas as pd 
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout, Bidirectional
from tensorflow.keras import regularizers
from tensorflow.keras.losses import sparse_categorical_crossentropy

from tqdm.notebook import trange
from IPython.display import Image, display

from big_sleep import Imagine

# Load pre-trained data from Google Drive 
Pre-trained (70 Epochs) Models for Chicken, Meat and Fish recipes.


In [ ]:
! gdown -- 1TtC9S4b5KSGcDeVaLzaciAr7rCwO4Bgv # https://drive.google.com/file/d/1TtC9S4b5KSGcDeVaLzaciAr7rCwO4Bgv/view?usp=sharing # model for chicken dish 

! gdown -- 1s9Tw0gOfaNnfZTS3stDp6lpk4UeqfuAo # https://drive.google.com/file/d/1s9Tw0gOfaNnfZTS3stDp6lpk4UeqfuAo/view?usp=sharing # model for meat dish

! gdown -- 1h6xCW_gpfunhAmXZ9TonC6Xr-Q_Nz7Tz # https://drive.google.com/file/d/1h6xCW_gpfunhAmXZ9TonC6Xr-Q_Nz7Tz/view?usp=sharing # model for fish dish

Downloading...
From: https://drive.google.com/uc?id=1TtC9S4b5KSGcDeVaLzaciAr7rCwO4Bgv
To: /content/chicken_model.h5
100% 164M/164M [00:02<00:00, 66.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1s9Tw0gOfaNnfZTS3stDp6lpk4UeqfuAo
To: /content/meat_model.h5
100% 164M/164M [00:02<00:00, 59.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1h6xCW_gpfunhAmXZ9TonC6Xr-Q_Nz7Tz
To: /content/fish_model.h5
100% 164M/164M [00:01<00:00, 108MB/s]


#Data Cleaning

In [ ]:
import pandas as pd

# I did not get the data from Kaggle directly since it required API token, which would be a problem for the reader/user. 
# Therefore, I upload the raw data to my GitHub repository and call it from here.

url = 'https://raw.githubusercontent.com/enesbasbug/Generate_image_from_generated_text/main/data_recipe.csv'
 # just in case, https://github.com/enesbasbug/Food_Recipe_Generator/blob/main/data/data_recipe.csv
df = pd.read_csv(url, index_col=0)
df.head(2)

,category,cooking_method,cuisine,image,ingredients,prep_time,recipe_name,serves,tags
0,NaN,['Set the racks in the middle and upper thirds...,['American'],https://www.skinnytaste.com/wp-content/uploads...,"['1 tablespoons extra virgin olive oil', '1 cu...",20 minutes,Mediterranean Sea Bass,4 servings,"Dairy Free, Gluten Free, Keto Recipes, Kid Fri..."
1,NaN,['Place the eggs in the air fryer basket and c...,['American'],https://www.skinnytaste.com/wp-content/uploads...,"['4 large eggs', 'Salt (black pepper, everythi...",15 minutes,Air Fryer Hard Boiled Eggs,4 eggs,"Air Fryer Recipes, Dairy Free, Gluten Free, Ke..."


In [ ]:
# Removing unnecessary features of the data
df.drop(columns=['category', 'cuisine', 'image', 'prep_time', 'serves', 'tags', 'recipe_name'], inplace=True)

In [ ]:
df.head(2)

,cooking_method,ingredients
0,['Set the racks in the middle and upper thirds...,"['1 tablespoons extra virgin olive oil', '1 cu..."
1,['Place the eggs in the air fryer basket and c...,"['4 large eggs', 'Salt (black pepper, everythi..."


In [ ]:
# Cleaning the data
for i in df.columns:
    df[i].replace('[]', np.nan, inplace=True)

for i in df.columns:
    df.dropna(subset=[i], inplace=True)

df = df.reset_index(drop=True)

In [ ]:
# Categorizing the date base on its type of meat such as chicken, meat and fish
df['chicken'] = [1 if 'chicken' in s.lower() else 0 for s in df['ingredients']]

df['meat'] = [1 if ('beef' or 'meat' or 'pork' or 'bacon') in s.lower() else 0 for s in df['ingredients']]
                            
df['fish'] = [1 if ('fish' or 'salmon' or 'sea bass') in s.lower() else 0 for s in df['ingredients']]

In [ ]:
print(len(df.chicken))
print(len(df.meat))
print(len(df.fish))

56312
56312
56312


In [ ]:
import re
from string import punctuation
# I wanted to keep ',' and '.' in the text so that the generated text/recipe seems real, accurate and easy to read.
# The only punctuations the text have dot and comma.
my_punctuation = punctuation.replace(".","")
my_punctuation = my_punctuation.replace(",","")


c = [df['cooking_method'][i].lower() for i in range(len(df)) if (df['chicken'][i] == 1)]
c = ''.join(c)
chicken = c.translate(str.maketrans("", "", my_punctuation))

m = [df['cooking_method'][i].lower() for i in range(len(df)) if (df['meat'][i] == 1)]
m = ''.join(m)
meat = m.translate(str.maketrans("", "", my_punctuation))

f = [df['cooking_method'][i].lower() for i in range(len(df)) if (df['fish'][i] == 1)]
f = ''.join(f)
fish = f.translate(str.maketrans("", "", my_punctuation))

ingredients = [df['ingredients'][i].lower() for i in range(len(df))]
ingredients = ''.join(f)
ingredients = f.translate(str.maketrans("", "", punctuation))



In [ ]:
# I limited the number of characters in the text files
fish = fish[:500000]
chicken = chicken[:500000]
meat = meat[:500000]
ingredients = ingredients[:10000]

In [ ]:
print(len(fish))
print(len(meat))
print(len(fish))
print(len(ingredients))

500000
500000
500000
10000


In [ ]:
file_f = open("fish.txt","w")#write mode
file_f.write(fish)
file_f.close()

file_m = open("meat.txt","w")#write mode
file_m.write(meat)
file_m.close()

file_c = open("chicken.txt","w")#write mode
file_c.write(chicken)
file_c.close()

file_i = open("ingredients.txt","w")#write mode
file_i.write(ingredients)
file_i.close()

# Reading the data

In [ ]:
TEXT_chicken = open('chicken.txt', 'rb').read().decode(encoding='utf-8')
TEXT_meat = open('meat.txt', 'rb').read().decode(encoding='utf-8')
TEXT_fish = open('fish.txt', 'rb').read().decode(encoding='utf-8')
TEXT_ingredients = open('ingredients.txt', 'rb').read().decode(encoding='utf-8')
# checking the number of characters in it
print(len(TEXT_chicken))
print(len(TEXT_meat))
print(len(TEXT_fish))
print(len(TEXT_ingredients))

500000
500000
500000
10000


# Common functions that recipe generation model uses

In [ ]:

def split_input_target(chunk):
    input_TEXT = chunk[:-1]
    target_TEXT = chunk[1:]
    return input_TEXT, target_TEXT

def loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim,batch_input_shape=[batch_size, None]))
    model.add(LSTM(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    model.add(Dropout(0.2))
    model.add(LSTM(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    model.add(Dropout(0.2))
    
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=loss) 
    # print(model.summary())
    return model

def generate_text(model, start_string,t):
    
    # Number of characters to generate
    num_generate = 1000
    # vectorizing - Converting our start string to numbers 
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    # Empty string to store our results
    text_generated = []
    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = t
    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    return (start_string + ''.join(text_generated))


# Tring to find items in the recipes; however, it was not that successfull due to the
# the ingredients in the data was long terms instead of one words. Therefore, I could not
# compare the items in the generated recipe with the items in the ingrediens text file.
# for instance, in the ingredients data there was '1 spoon of olive oil' instead of just 'olive oil'
# so that i could not extract the items in generated recipe properly.
def word_extraction(text):
  ignore = ["a", "the", "is", "to", "until","about","with","and","as","add","on","in","at","of","over","around","knife","sharp","pairing","once","comes","for","minutes","hours"]
  words = re.sub("[^\w]", " ",  text).split()
  cleaned_text = [w.lower() for w in words if w not in ignore]
  output_ingredients = ["".join(a) for a in cleaned_text if a in TEXT_ingredients]
  return output_ingredients


BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000
# The embedding dimension
embedding_dim = 256
# Number of RNN units
rnn_units = 1024
EPOCHS = 70

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)



# Generating Fish Food Recipe

In [ ]:
# The unique characters in the file
vocab = sorted(set(TEXT_fish))

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
TEXT_as_int = np.array([char2idx[c] for c in TEXT_fish])
seq_length = 100

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(TEXT_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Length of the vocabulary in chars
vocab_size = len(vocab)
model = build_model(vocab_size=len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)



# history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
# model.save("fish_model.h5")

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights("fish_model.h5")
# model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
fish_recipe = generate_text(model, start_string=u"fish and ",t=0.1)
# fish_recipe

In [ ]:
fish_recipe

'fish and remove half of the satay for serving. add about 1 cup of the broth to the remaining satay in the skillet and bring to a simmer. cook 5 to 8 minutes, or until the fish is cooked through.in a bowl, mix the marinade ingredients except the chicken. slice the chicken into long, thin strips, about 3 inches long by 1 inch wide, and add to marinade. turn chicken to coat, cover and refrigerate at least 4 hours or longer., meanwhile, make the dressing whisk the lime juice, oil, salt and pepper in a food processor fitted with a metal blade. pulse to combine well. using a rubber spatula, heap the mixture onto a bowl and set aside., for the seasoned flour whisk together the allpurpose flour and cook, stirring often, until all the vegetables are softened, but not cooked through. add the cooked rice, soy sauce, fish sauce, thai basil, mint leaves, and cilantro and toss until thoroughly coated., for the blackened catfish tacos heat a saute pan on medium heat, stirring occasionally, until the

In [ ]:
file_ff = open("generated_fish_recipe.txt","w")#write mode
file_ff.write(fish_recipe)
file_ff.close()

In [ ]:
# To generate something new please remove the #s below
# the beginning of the recipe look like each other sometimes
# however the rest of it would definitely change.
# ___________________________

# text_file_f = generate_text(model, start_string=u"fish and ",t=0.1)
# text_file_f

'fish and bake it in a preheated 375 degree f oven, covered with foil, until the fish just flakes about 15 minutes., with a slotted spoon, transfer meat to a platter. reduce heat to medium low and cook shallots until well browned, about 15 minutes. add garlic and ginger stir fry 30 seconds to 1 minute per side. transfer to a medium bowl and cover with hot water let sit an additional 2 minutes., place a spoonful of green papaya slaw on the side and arrange two sausages on top. garnish the plate with a few drops of harissa sauce and a sprinkle of peanuts, and serve hot., in a small pot, combine the sugar, garlic, lime juice, and fish sauce and simmer until the clams and mussels have opened., for the rouille broil the tomato and bell pepper, turning to blacken and stir until it starts thickening to almost a light syrup consistency. once it has thickened add crab base and make sure it is incorporated well. then add 23 cup heavy whipping cream stir over medium heat until it is reduced by 12

# Generating Meat Food Recipe

In [ ]:
# The unique characters in the file
vocab = sorted(set(TEXT_meat))

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
TEXT_as_int = np.array([char2idx[c] for c in TEXT_meat])
seq_length = 100

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(TEXT_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Length of the vocabulary in chars
vocab_size = len(vocab)

model = build_model(vocab_size=len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)
    

# history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
# model.save("meat_model.h5")
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights("meat_model.h5")
# model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
meat_recipe = generate_text(model, start_string=u"meat and ",t=0.1)
# meat_recipe

In [ ]:
meat_recipe

'meat and add the remaining teaspoon of cumin., heat the taco shells according to the directions, then assemble placing 14 cup beef in each shell, topped with lettuce, 1 tablespoon cheese and tomato., instead of foil with the meat and salt, cook breaking the meat up into small pieces as it cooks. when browned, add the onion, celery and carrots and saute 3 to 4 minutes., transfer the meat and vegetables to a 2quart oval casserole dish and spread the mashed potatoes over the top, leaving a 14inch boarder around the edge. make a decorative pattern on the top of the potatoes, if desired. sprinkle with cheese, if using, and dot with the remaining tablespoon of butter. bake until the crust is golden and crisp., remove from oven and drain on a cutting board. serve with ketchup and mustard for dipping.instant pot, assuming your eles f. combine the mustard, vinegar, and salt in a small bowl. spread the mixture on the filet and brush it around the beef reads 130f. cover the pan and refrigerate f

In [ ]:
file_mm = open("generated_meat_recipe.txt","w")#write mode
file_mm.write(meat_recipe)
file_mm.close()

In [ ]:
# To generate something new please remove the #s below
# the beginning of the recipe look like each other sometimes
# however the rest of it would definitely change.
# ___________________________

# text_file_m = generate_text(model, start_string=u"meat and ",t=0.1)
# text_file_m

'meat and add the olive oil. season the beef tenderloin with salt, and pepper, and sear on all sides until a nice crust is formed, 1 to 2 minutes per side. add the thyme, garlic, buttermilk or yogurt, mustard, bread crumbs, salt and pepper in a food processor fitted with a metal blade, pulse the egg yolks on low. with the machine remaining ingredients. 4.divide empanadas among baking sheets, leaving 112 space between each empanadas with remaining egg wash. cook until pastries puff, turn golden brown and the gratin in bubbling., in a bowl, stir together the water, some oil and 1 tablespoon of the salt in a small bowl. spread the mixture on the filet and brush it around the beef reads 130f. cover the pan and refrigerate for 1 hour. 2.reheat the oven to 425f. beat the tortillas in the microwave for 1 minute. for medium rare 145°f to medium 160°f doneness, turning occasionally. 3season steak with salt and pepper, as desired. carve steaks into thin strips. top tortillas evenly with steak, q

#Generatin Chicken Food Recipe

In [ ]:
# The unique characters in the file
vocab = sorted(set(TEXT_chicken))

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
TEXT_as_int = np.array([char2idx[c] for c in TEXT_chicken])
seq_length = 100

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(TEXT_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Length of the vocabulary in chars
vocab_size = len(vocab)

model = build_model(vocab_size=len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)

# history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
# model.save("chicken_model.h5")
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights("chicken_model.h5")
# model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
chicken_recipe = generate_text(model, start_string=u"chicken and ",t=0.1)

# chicken_recipe

In [ ]:
chicken_recipe

'chicken and bring to a boil, then partially cover, simmer on low 30 minutes., meanwhile, cook cavatella in a small bowl and lay out the plantains with the other tortilla., cheese should be melted and the bottom of the tortillas on a microwave safe dish and microwave 40 to 50 seconds to make them pliable so they dont crack. keep them covered for 710 minutes. set aside until ready to use. makes 4 cups.preheat oven to 375 degrees., in a large skillet, melt butter over mediumlow heat and add chicken. cook on both sides, until golden brown and the cheese is melted and gooey in the center, 3 to 5 minutes per side. transfer to paper towels to drain let cool slightly, then top the enchiladas with sour cream, jalapenos, red chile pepper, cilantro and jalapeno slices if desired. serve over lettuce leaves.preheat the oven to 450 degrees f. set a wire rack on a rimmed baking sheet. when the oil is hot, add the onions, bell peppers, and remaining 12 tsp cumin, 12 tsp paprika, 14 tsp chili powder, 

In [ ]:
file_cc = open("generated_chicken_recipe.txt","w")#write mode
file_cc.write(chicken_recipe)
file_cc.close()

In [ ]:
# To generate something new please remove the #s below
# the beginning of the recipe look like each other sometimes
# however the rest of it would definitely change.
# ___________________________

# text_file_c = generate_text(model, start_string=u"chicken and ",t=0.1)
# text_file_c

'chicken and bring to a boil, then partially cover, simmer on low 30 minutes., meanwhile, cook cavatella in a small bowl and lower the sauce over the meat.preheat the oven to 400f., finely chop half of the cremini mushrooms and transfer to a plate., add the oil to the skillet, brussels sprouts, shallots and saute until they are starting to caramelize, about 5 to 6 minutes. deglaze with 12 cup water, bay leaves and 18 teaspoon black pepper., cover and cook high pressure 15 minutes, natural release., remove bay leaf, adjust salt and pepper and enjoy,  saute about 2 to 3 minutes until soft then add the tomatoes and saute another minute, until soft., add zucchini and season with salt and pepper. cook about 15 minutes, or until the zucchini is soft and cooked through., air fryer method, follow steps 15., preheat the air fryer once again to 400f. spray both sides of the chicken with oil. cook, in two batches 12 minutes turning halfway.season the pork with salt, garlic powder and fresh cracke

# Code for finding Inredients in the generated recipe

In [ ]:
meat_recipe[:50]

'meat and add the remaining teaspoon of cumin., heat the taco shells according to the directions, then assemble placing 14 cup beef in each shell, topped with lettuce, 1 tablespoon cheese and tomato., instead of foil with the meat and salt, cook breaking the meat up into small pieces as it cooks. when browned, add the onion, celery and carrots and saute 3 to 4 minutes., transfer the meat and vegetables to a 2quart oval casserole dish and spread the mashed potatoes over the top, leaving a 14inch boarder around the edge. make a decorative pattern on the top of the potatoes, if desired. sprinkle with cheese, if using, and dot with the remaining tablespoon of butter. bake until the crust is golden and crisp., remove from oven and drain on a cutting board. serve with ketchup and mustard for dipping.instant pot, assuming your eles f. combine the mustard, vinegar, and salt in a small bowl. spread the mixture on the filet and brush it around the beef reads 130f. cover the pan and refrigerate f

In [ ]:
# Tring to find items in the recipes; however, it was not that successfull due to the
# the ingredients in the raw data was long terms instead of one words. Therefore, I could not
# compare the items in the generated recipe with the items in the ingrediens text file(raw data)
# for instance, in the ingredients data there was '1 spoon of olive oil' instead of just 'olive oil'
# so that i could not extract the items in generated recipe properly.

# word_extraction function is defined in the Common functions above
ingredients_m = word_extraction(meat_recipe)
ingredients_c = word_extraction(chicken_recipe)
ingredients_f = word_extraction(fish_recipe)
# ## ingredients of new meat recipe
print(ingredients_m)

['meat', 'remaining', 'teaspoon', 'heat', 'according', 'directions', 'then', '14', 'cup', 'beef', 'each', 'topped', 'lettuce', '1', 'tablespoon', 'cheese', 'tomato', 'instead', 'meat', 'salt', 'cook', 'breaking', 'meat', 'up', 'into', 'small', 'pieces', 'it', 'when', 'browned', 'onion', 'saute', '3', '4', 'transfer', 'meat', 'vegetables', 'top', 'edge', 'make', 'top', 'if', 'desired', 'sprinkle', 'cheese', 'if', 'using', 'dot', 'remaining', 'tablespoon', 'butter', 'bake', 'golden', 'remove', 'from', 'oven', 'serve', 'dipping', 'pot', 'your', 'f', 'combine', 'vinegar', 'salt', 'small', 'bowl', 'mixture', 'it', 'beef', 'cover', 'pan', 'refrigerate', '1', 'hour']


# Text to Image Generator using Big Sleep

**generated images will appear in the folder where it will be saved on the left, in order to see them please check there**

In [ ]:
text_to_img_fish = fish_recipe[:210]
text_to_img_meat = meat_recipe[:210]
text_to_img_chicken = chicken_recipe[:210]
mix = [text_to_img_fish, text_to_img_meat, text_to_img_chicken]
for T in range(3):
  TEXT = mix[T]
  SAVE_EVERY = 100 
  SAVE_PROGRESS = True 
  LEARNING_RATE = 5e-2 
  ITERATIONS = 1050 
  SEED = 0 
  model = Imagine(
      text = TEXT,
      save_every = SAVE_EVERY,
      lr = LEARNING_RATE,
      iterations = ITERATIONS,
      save_progress = SAVE_PROGRESS,
      seed = SEED
  )
  for epoch in trange(1, desc = 'epochs'): # number of epochs
      for i in trange(300, desc = 'iteration'): # you can decrease 'the iteration' to achieve faster results
          model.train_step(epoch, i)

          if i == 0 or i % model.save_every != 0:
              continue
          filename = TEXT.replace(' ', '_')
          image = Image(f'./{filename}.png')
          display(image)

setting seed of 0
you can override this with --seed argument in the command line, or --random for a randomly chosen one


100%|████████████████████████████████████████| 354M/354M [00:01<00:00, 250MiB/s]
100%|██████████| 800/800 [00:00<00:00, 390122.45B/s]


epochs:   0%|          | 0/1 [00:00<?, ?it/s]

iteration:   0%|          | 0/300 [00:00<?, ?it/s]

image updated at "./fish_and_remove_half_of_the_satay_for_serving._add_about_1_cup_of_the_broth_to_the_remaining_satay_in_the_skillet_and_bring_to_a_simmer._cook_5_to_8_minutes_or_until_the_fish_is_cooked_through.in_a_bowl_mix_t.png"


...

epochs:   0%|          | 0/1 [00:00<?, ?it/s]

iteration:   0%|          | 0/300 [00:00<?, ?it/s]

image updated at "./chicken_and_bring_to_a_boil_then_partially_cover_simmer_on_low_30_minutes._meanwhile_cook_cavatella_in_a_small_bowl_and_lay_out_the_plantains_with_the_other_tortilla._cheese_should_be_melted_and_the_bottom.png"


image updated at "./chicken_and_bring_to_a_boil_then_partially_cover_simmer_on_low_30_minutes._meanwhile_cook_cavatella_in_a_small_bowl_and_lay_out_the_plantains_with_the_other_tortilla._cheese_should_be_melted_and_the_bottom.png"


image updated at "./chicken_and_bring_to_a_boil_then_partially_cover_simmer_on_low_30_minutes._meanwhile_cook_cavatella_in_a_small_bowl_and_lay_out_the_plantains_with_the_other_tortilla._cheese_should_be_melted_and_the_bottom.png"


In [ ]:
# Reminder
# You may also visit my GitHub https://github.com/enesbasbug/Food_Recipe_Generator 
# repository to see many texts/images/gifs examples of my project.

In [ ]:
print('Thank you!')

Thank you!
